In [1]:
# Import Dependencies 
import pandas as pd
import hvplot.pandas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [3]:
# Read in Health Ranking Data
df = pd.read_csv("Data/County_Health_All_Factors.csv")
print(df.shapee)
df.head()

(3142, 10)


,Unnamed: 0,FIPS,State,County,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank
0,1,1001,Alabama,Autauga,8,5,15,14,5,50
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62
2,3,1005,Alabama,Barbour,14,48,53,30,61,32
3,4,1007,Alabama,Bibb,47,24,38,16,38,31
4,5,1009,Alabama,Blount,36,14,10,41,15,53


In [8]:
# Read in the Merged Data
df2 = pd.read_csv("Data/Merged_Demographic_Data.csv")
print(df2.shape)
df2.head()

(3022, 26)


,Unnamed: 0,County,State,FIPS,Cases2019,Population,% < 18 Yrs Old,% 65 Yrs Old and over,Income ($),African American (Count),...,Native Hawaiian/Other Pacific Islander (%),Hispanic (Count),Hispanic (%),Non-Hispanic White (Count),Non-Hispanic White (%),Female (%),Rural (Count),Rural (%),Life Expectancy,Deaths (Count)
0,0,Autauga,Alabama,1001,1,55504.0,23.937014,15.119631,58343.0,10687.0,...,0.104497,1586.0,2.857452,41336.0,74.473912,51.342246,22921.0,42.002162,76.330589,815.0
1,1,Baldwin,Alabama,1003,0,212628.0,21.848487,19.947044,56607.0,19037.0,...,0.068665,9675.0,4.550200,176582.0,83.047388,51.452772,77060.0,42.279099,78.599498,2827.0
2,2,Barbour,Alabama,1005,1,25270.0,20.763751,18.824693,32490.0,12115.0,...,0.185991,1063.0,4.206569,11613.0,45.955679,47.229917,18613.0,67.789635,75.779457,451.0
3,3,Bibb,Alabama,1007,0,22668.0,20.606141,16.022587,45795.0,4864.0,...,0.114699,598.0,2.638080,16842.0,74.298571,46.453150,15663.0,68.352607,73.928271,445.0
4,4,Blount,Alabama,1009,0,58013.0,23.349939,17.842553,48253.0,847.0,...,0.117215,5549.0,9.565097,50439.0,86.944306,50.688639,51562.0,89.951502,74.597767,1050.0


In [15]:
df_merged = df.merge(df2, how='left', on='FIPS')
print(df_merged.shape)
df_merged.head(10)

(3150, 35)


,Unnamed: 0_x,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Native Hawaiian/Other Pacific Islander (%),Hispanic (Count),Hispanic (%),Non-Hispanic White (Count),Non-Hispanic White (%),Female (%),Rural (Count),Rural (%),Life Expectancy,Deaths (Count)
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,0.104497,1586.0,2.857452,41336.0,74.473912,51.342246,22921.0,42.002162,76.330589,815.0
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,0.068665,9675.0,4.550200,176582.0,83.047388,51.452772,77060.0,42.279099,78.599498,2827.0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,0.185991,1063.0,4.206569,11613.0,45.955679,47.229917,18613.0,67.789635,75.779457,451.0
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,0.114699,598.0,2.638080,16842.0,74.298571,46.453150,15663.0,68.352607,73.928271,445.0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,0.117215,5549.0,9.565097,50439.0,86.944306,50.688639,51562.0,89.951502,74.597767,1050.0
5,6,1011,Alabama,Bullock,53,58,65,62,64,24,...,0.776021,850.0,8.245223,2196.0,21.301775,45.533029,5607.0,51.374382,73.117530,205.0
6,7,1013,Alabama,Butler,63,56,56,52,55,8,...,0.050441,285.0,1.437579,10229.0,51.596469,53.432535,14921.0,71.232157,73.505385,393.0
7,8,1015,Alabama,Calhoun,55,10,28,34,27,41,...,0.112440,4302.0,3.749739,82961.0,72.311031,51.930653,39955.0,33.696826,73.102948,2333.0
8,9,1017,Alabama,Chambers,42,53,57,25,37,46,...,0.044493,822.0,2.438229,18710.0,55.497879,52.125293,16816.0,49.148034,74.122278,691.0
9,10,1019,Alabama,Cherokee,43,8,12,54,19,43,...,0.030939,425.0,1.643655,23707.0,91.685037,50.319063,22282.0,85.736273,74.346572,575.0


In [18]:
for col in df_merged.columns:
    print(col)

Unnamed: 0_x
FIPS
State_x
County_x
Lifespan_Rank
Life_Quality_Rank
Health_Behaviors_Rank
Clinical_Care_Rank
Social_Economic_Factors_Rank
Physical_Environment_Rank
Unnamed: 0_y
County_y
State_y
Cases2019
Population
% < 18 Yrs Old
% 65 Yrs Old and over
Income ($)
African American (Count)
African American (%)
American Indian/Alaskan Native (Count)
American Indian/Alaskan Native(%)
Asian (Count)
Asian (%)
Native Hawaiian/Other Pacific Islander (Count)
Native Hawaiian/Other Pacific Islander (%)
Hispanic (Count)
Hispanic (%)
Non-Hispanic White (Count)
Non-Hispanic White (%)
Female (%)
Rural (Count)
Rural (%)
Life Expectancy
Deaths (Count)


In [16]:
df_merged.dtypes

Unnamed: 0_x                                        int64
FIPS                                                int64
State_x                                            object
County_x                                           object
Lifespan_Rank                                      object
Life_Quality_Rank                                  object
Health_Behaviors_Rank                              object
Clinical_Care_Rank                                 object
Social_Economic_Factors_Rank                       object
Physical_Environment_Rank                          object
Unnamed: 0_y                                      float64
County_y                                           object
State_y                                            object
Cases2019                                         float64
Population                                        float64
% < 18 Yrs Old                                    float64
% 65 Yrs Old and over                             float64
Income ($)    